# Reading from CSVs and Other Delimited Files

In [2]:
import pandas as pd
pd.__version__

'1.3.2'

In [17]:
import pandas as pd
from pathlib import Path

In [126]:
filepath = Path('../../datasets/Ch2/movieboxoffice.csv')


In [129]:
ts = pd.read_csv(filepath,
                 header=0,
                 parse_dates=[0],
                 index_col=0,
                 infer_datetime_format=True,
                 usecols=['Date',
                          'DOW',
                          'Daily',
                          'Forecast',
                          'Percent Diff'])
ts.head()

,DOW,Daily,Forecast,Percent Diff
Date,,,,
2021-04-26,Friday,"$125,789.89","$235,036.46",-46.48%
2021-04-27,Saturday,"$99,374.01","$197,622.55",-49.72%
2021-04-28,Sunday,"$82,203.16","$116,991.26",-29.74%
2021-04-29,Monday,"$33,530.26","$66,652.65",-49.69%
2021-04-30,Tuesday,"$30,105.24","$34,828.19",-13.56%


In [42]:
ts.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 128 entries, 2021-04-26 to 2021-08-31
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   DOW           128 non-null    object
 1   Daily         128 non-null    object
 2   Forecast      128 non-null    object
 3   Percent Diff  128 non-null    object
dtypes: object(4)
memory usage: 5.0+ KB


In [45]:
clean = lambda x: x.str.replace('[^\d]', '', regex=True)                                                      
c_df = ts[['Daily', 'Forecast']].apply(clean, axis=1)
ts[['Daily', 'Forecast']] = c_df.astype(float)
                                

In [46]:
ts.head()

,DOW,Daily,Forecast,Percent Diff
Date,,,,
2021-04-26,Friday,12578989.0,23503646.0,-46.48%
2021-04-27,Saturday,9937401.0,19762255.0,-49.72%
2021-04-28,Sunday,8220316.0,11699126.0,-29.74%
2021-04-29,Monday,3353026.0,6665265.0,-49.69%
2021-04-30,Tuesday,3010524.0,3482819.0,-13.56%


In [57]:
ts.memory_usage()

Index           1024
DOW             1024
Daily           1024
Forecast        1024
Percent Diff    1024
dtype: int64

In [47]:
ts.memory_usage().sum()

5120

## Using `date_parser`

In [56]:
date_parser = lambda x: pd.to_datetime(x, format="%d-%b-%y")
ts = pd.read_csv(filepath,
                 parse_dates=[0],
                 index_col=0,
                 date_parser=date_parser,
                 usecols=[0,1,3, 7, 6])
ts.head()

,DOW,Daily,Forecast,Percent Diff
Date,,,,
2021-04-26,Friday,"$125,789.89","$235,036.46",-46.48%
2021-04-27,Saturday,"$99,374.01","$197,622.55",-49.72%
2021-04-28,Sunday,"$82,203.16","$116,991.26",-29.74%
2021-04-29,Monday,"$33,530.26","$66,652.65",-49.69%
2021-04-30,Tuesday,"$30,105.24","$34,828.19",-13.56%


# Reading data from an Excel file

In [58]:
import pandas as pd
from pathlib import Path
filepath = Path('../../datasets/Ch2/sales_trx_data.xlsx')

In [59]:
excelfile = pd.ExcelFile(filepath)
excelfile.sheet_names

['2017', '2018']

In [60]:
ts = pd.read_excel(filepath,
                    engine='openpyxl',
                    index_col=1,
                    sheet_name=[0,1],
                    parse_dates=True)
ts.keys()

dict_keys([0, 1])

In [61]:
ts = pd.read_excel(filepath,
                    engine='openpyxl',
                    index_col=1,
                    sheet_name=['2017','2018'],
                    parse_dates=True)
ts.keys()

dict_keys(['2017', '2018'])

In [62]:
ts = pd.read_excel(filepath,
                    engine='openpyxl',
                    index_col=1,
                    sheet_name=None,
                    parse_dates=True)
ts.keys()

dict_keys(['2017', '2018'])

In [63]:
ts_combined = pd.concat([ts['2017'],ts['2018']])

In [64]:
ts_combined.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 74124 entries, 2017-01-01 to 2018-12-31
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Line_Item_ID        74124 non-null  int64 
 1   Credit_Card_Number  74124 non-null  int64 
 2   Quantity            74124 non-null  int64 
 3   Menu_Item           74124 non-null  object
dtypes: int64(3), object(1)
memory usage: 2.8+ MB


In [65]:
ts_combined = pd.concat(ts).droplevel(level=0)
ts_combined.head()

,Line_Item_ID,Credit_Card_Number,Quantity,Menu_Item
Date,,,,
2017-01-01,1,7437926611570790,1,spicy miso ramen
2017-01-01,2,7437926611570790,1,spicy miso ramen
2017-01-01,3,8421920068932810,3,tori paitan ramen
2017-01-01,4,8421920068932810,3,tori paitan ramen
2017-01-01,5,4787310681569640,1,truffle butter ramen


In [66]:
ts = pd.read_excel(filepath,
                   index_col=1,
                   sheet_name='2018',
                   parse_dates=True)
type(ts)

pandas.core.frame.DataFrame

## Using `ExcelFile`

In [69]:
excelfile = pd.ExcelFile(filepath)
excelfile.parse(sheet_name='2017',
                index_col=1,
                parse_dates=True).head()


,Line_Item_ID,Credit_Card_Number,Quantity,Menu_Item
Date,,,,
2017-01-01,1,7437926611570790,1,spicy miso ramen
2017-01-01,2,7437926611570790,1,spicy miso ramen
2017-01-01,3,8421920068932810,3,tori paitan ramen
2017-01-01,4,8421920068932810,3,tori paitan ramen
2017-01-01,5,4787310681569640,1,truffle butter ramen


# Reading data from a URL

In [70]:
import pandas as pd

## Reading from GitHub

In [72]:
# example of produced error
url = 'https://github.com/PacktPublishing/Time-Series-Analysis-with-Python-Cookbook./blob/main/datasets/Ch2/AirQualityUCI.csv'
# pd.read_csv(url)


In [73]:
url = 'https://raw.githubusercontent.com/PacktPublishing/Time-Series-Analysis-with-Python-Cookbook./main/datasets/Ch2/AirQualityUCI.csv'
df = pd.read_csv(url,
                 delimiter=';',
                 index_col='Date',
                 parse_dates=True)

df.iloc[:3,1:4]


,CO(GT),PT08.S1(CO),NMHC(GT)
Date,,,
2004-10-03,"2,6",1360.0,150.0
2004-10-03,2,1292.0,112.0
2004-10-03,"2,2",1402.0,88.0


## Reading from Public S3 Bucket

In [74]:
url = 'https://tscookbook.s3.amazonaws.com/AirQualityUCI.xlsx'
df = pd.read_excel(url,
                   index_col='Date',
                   parse_dates=True)


In [76]:
s3uri = 's3://tscookbook/AirQualityUCI.xlsx'
df = pd.read_excel(s3uri,
                   index_col='Date',
                   parse_dates=True)

## Reading from Private S3 Bucket

In [78]:
import configparser
config = configparser.ConfigParser()
config.read('aws.cfg')

AWS_ACCESS_KEY = config['AWS']['aws_access_key']
AWS_SECRET_KEY = config['AWS']['aws_secret_key']


In [ ]:
s3uri = "s3://tscookbook-private/AirQuality.csv"

df = pd.read_csv(s3uri,
                 index_col='Date',
                 parse_dates=True,
                 storage_options= {
                         'key': AWS_ACCESS_KEY,
                         'secret': AWS_SECRET_KEY
                     })


**Using BOTO3**

In [ ]:
import boto3
bucket = "tscookbook-private"
client = boto3.client("s3",
                  aws_access_key_id =AWS_ACCESS_KEY,
                  aws_secret_access_key = AWS_SECRET_KEY)


In [ ]:
data = client.get_object(Bucket=bucket, Key='AirQuality.csv')
df = pd.read_csv(data['Body'],
                 index_col='Date',
                 parse_dates=True)


## Reading from HTML

In [79]:
url = "https://en.wikipedia.org/wiki/COVID-19_pandemic_by_country_and_territory"
results = pd.read_html(url)
print(len(results))


58


In [97]:
df = results[13]
df.columns

Index(['2020', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep',
       'Oct', 'Nov', 'Dec'],
      dtype='object')

In [98]:
df = df.iloc[4:].T
df.columns = df.iloc[0]
df.drop('2020', inplace=True)
df[['USA', 'Canada', 'UK', 'China']].tail()


2020,USA,Canada,UK,China
Aug,151265.0,8929.0,46119.0,4668.0
Sep,182162.0,9117.0,41501.0,4730.0
Oct,204642.0,9291.0,42143.0,4746.0
Nov,228185.0,10110.0,46555.0,4746.0
Dec,264808.0,12032.0,58448.0,4750.0


## Example how `read_html()` works

In [99]:
import pandas as pd
html = """
    ...: <table>
    ...:   <tr>
    ...:     <th>Ticker</th>
    ...:     <th>Price</th>
    ...:   </tr>
    ...:   <tr>
    ...:     <td>MSFT</td>
    ...:     <td>230</td>
    ...:   </tr>
    ...:   <tr>
    ...:     <td>APPL</td>
    ...:     <td>300</td>
    ...:   </tr>
    ...:     <tr>
    ...:     <td>MSTR</td>
    ...:     <td>120</td>
    ...:   </tr>
    ...: </table>
    ...:
    ...: </body>
    ...: </html>
    ...: """

df = pd.read_html(html)
df[0]


,Ticker,Price
0,MSFT,230
1,APPL,300
2,MSTR,120


## Using `attr` option in `read.html()`

In [100]:
import pandas as pd
url = "https://en.wikipedia.org/wiki/COVID-19_pandemic_by_country_and_territory"
df = pd.read_html(url, attrs={'id': 'thetable'})
len(df)


4

In [101]:
df[2].columns

Index(['2020', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep',
       'Oct', 'Nov', 'Dec'],
      dtype='object')

# Reading from a SAS dataset

In [110]:
import pandas as pd
path = '../../datasets/Ch2/DCSKINPRODUCT.sas7bdat'


In [111]:
df = pd.read_sas(path, chunksize=10000)
type(df)


pandas.io.sas.sas7bdat.SAS7BDATReader

In [112]:
results = []
for chunk in df:
    results.append(
        chunk.groupby('DATE')['Revenue']
             .agg(['sum', 'count']))
len(results)

16

In [113]:
results[0].loc['2013-02-10']

sum      923903.0
count        91.0
Name: 2013-02-10 00:00:00, dtype: float64

In [114]:
results[1].loc['2013-02-10']

sum      8186392.0
count         91.0
Name: 2013-02-10 00:00:00, dtype: float64

In [115]:
results[2].loc['2013-02-10']

sum      5881396.0
count         91.0
Name: 2013-02-10 00:00:00, dtype: float64

In [116]:
from functools import reduce
final = reduce(lambda x1, x2: x1.add(x2, fill_value=0), results)
type(final)

pandas.core.frame.DataFrame

In [117]:
final.loc['2013-02-10']

sum      43104420.0
count        1383.0
Name: 2013-02-10 00:00:00, dtype: float64

## Comparing Performance: Memory & Time

In [118]:
import pandas as pd
from functools import reduce

path = '../../datasets/Ch2/ahs2013n.sas7bdat'

def read_entire_data():
        dfs = pd.read_sas(path)
        final = dfs.groupby('REGION')['CONTROL'].count()
        return final


In [119]:
def chunk_and_reduce():
    dfs = pd.read_sas(path, chunksize=150000)
    for chunk in dfs:
        results.append(chunk.groupby('REGION')['CONTROL'].count())
        final = reduce(lambda x1, x2: 
                           x1.add(x2, fill_value=0), 
                       results)
        return final


In [120]:
def map_and_reduce():
    dfs = pd.read_sas(path, chunksize=150000)
    df_map = map(lambda a: 
                     a.groupby('REGION')['CONTROL'].count(), 
                 dfs)
    final = reduce(lambda x1, x2:
                       x1.add(x2, fill_value=0), 
                   df_map)
    return final


In [121]:
%load_ext memory_profiler

In [122]:
%%time
%memit read_entire_data()



peak memory: 6486.23 MiB, increment: 6265.21 MiB
CPU times: user 5min 47s, sys: 35.3 s, total: 6min 22s
Wall time: 6min 33s


In [123]:
%%time
%memit chunk_and_reduce()



peak memory: 6136.82 MiB, increment: 5954.84 MiB
CPU times: user 5min 28s, sys: 26.1 s, total: 5min 54s
Wall time: 5min 57s


In [124]:
%%time
%memit map_and_reduce()



peak memory: 6064.11 MiB, increment: 5880.79 MiB
CPU times: user 5min 25s, sys: 27.8 s, total: 5min 53s
Wall time: 5min 56s
